In [28]:
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import decomposition
from sklearn.preprocessing import StandardScaler
import glob
import os
from tslearn.clustering import TimeSeriesKMeans
import copy as cp
import random
import pickle
import multiprocessing
import time
from collections import Counter
import math
from sklearn.ensemble import IsolationForest
from collections import defaultdict, Counter

In [3]:
prefix ='/home/lab1/repo/planning/traj_pred/data/'
data=None
for i in range(3):
    name_te= prefix+'veh_test_set_'+str(i)+'.npy'
    te = np.load(name_te,allow_pickle=True).astype(float)
    if data is None:
        data=te
    else:
        data= np.vstack((data, te))
data=data[:,:,:2]
print(data.shape)

(153031, 10, 2)


In [25]:
def extract_feature(ele):
    """
    extract feature for a single trajectory
    f = [x, y, a,v, delta_a, delta_v]
    """
    fe=[]
    for i in range(3, len(ele)):
        prev3, prev2, prev, curr = ele[i-3], ele[i-2], ele[i-1], ele[i]
        v = np.linalg.norm(curr - prev)/0.5
        v_prev = np.linalg.norm(prev - prev2)/0.5
        v_prev2 = np.linalg.norm(prev2 - prev3)/0.5
        a = (v-v_prev)/0.5
        a_prev = (v_prev-v_prev2)/0.5
        
        energy = v**2-v_prev**2
        force = a-a_prev
        f = [ele[i][0], ele[i][1], v, a, energy, force]
        fe.append(f)
    return np.array(fe)

def isolation_f(feature):
    iso_f =[]
    for seq in feature:
        iso_f.append(seq[:,3].flatten())
    return np.array(iso_f)
        

    

In [24]:
feature = []
for ele in data:
    fe= extract_feature(ele)
    feature.append(fe)
feature=np.array(feature)
print(feature.shape)

(153031, 7, 6)


In [26]:
iso_feature = isolation_f(feature)
print(iso_feature.shape)

(153031, 7)


In [18]:
# curr=data[0][0]
# prev=data[0][1]
# print(curr, prev)
# print(math.sqrt((curr[0]-prev[0])**2+(curr[1]-prev[1])**2))
# print(np.linalg.norm(curr - prev))

[ 663.27612305 -397.70214844] [ 659.54040527 -393.78775024]
5.410924181593378
5.410924181593378


In [ ]:
def clustering(feature, k, metrics="dtw"):

    n_jobs=multiprocessing.cpu_count()
    print("start training clustering on",n_jobs, "cpus")
    start_time = time.time()
    km_dba = TimeSeriesKMeans(n_clusters=k, metric=metric, max_iter=30, max_iter_barycenter=5, n_jobs=-1, verbose=False, \
                                      random_state=np.random.randint(low=0,high=20,size=2)[0]).fit(feature)
    print("---used  %s seconds ---" % (time.time() - start_time))
    return km_dba


if __name__ == "__main__":
    if not os.path.exists("risk_model"):
        os.makedirs("risk_model")
#     if not os.path.exists("saved_centers"):
#         os.makedirs("saved_centers")

#     feature = np.load("data/Vehicle_trajectory.npy", allow_pickle=True)
    print(feature.shape)

    metric="dtw"
    n_initial = 10 # how many times to re-initial the run
    CENTERS=[]
    LABELS=[]
    k_min = 6
    k_max= 12
    elbow = []
    
    models={}
    for k in range(k_min, k_max+1):
        print("runing number of clusters:", k)
        best_in_a_k=[]
        for i in range(0,n_initial):
            print("running re-initialization:", i, "out of", n_initial)
            km_dba = clustering(feature, k, metrics=metric)
            CENTERS.append(km_dba.cluster_centers_)
            LABELS.append(km_dba.labels_)
            best_in_a_k.append(km_dba.inertia_)
#             pickle.dump(km_dba, open("saved_centers/cluster_model_k"+str(k)+"_i"+str(i)+".pkl", 'wb'))
        models[k]=km_dba
        elbow.append(min(best_in_a_k))
    pickle.dump(models, open("risk_model/models.pkl", 'wb'))
    pickle.dump(elbow, open("risk_model/elbow_list.pkl", 'wb'))
        
    plt.plot(range(k_min, k_max+1), elbow)
    plt.xlabel('Number of centroids')
    plt.ylabel('Elbow score')
    plt.title("Elbow score on vehicle features")
    plt.savefig("risk_model/elbow_plot.png")
    plt.show()

In [ ]:
### isolation forest
iso_clf = IsolationForest(n_estimators=100, contamination=0.3, behaviour='deprecated', warm_start=True)
iso_clf.fit(iso_feature)
pickle.dump(clf, open('risk_model/iforest_model.pkl', 'wb'))

In [1]:
############  run below after above is finished

In [ ]:
# compute prob after finding the best k and its model
k_best = 8 # <- manually select
kmeans =models[k_best]
total = Counter(kmeans.labels_)
clustering_labels = kmeans.labels_
labels = kmeans.predict(feature)
abnormal = defaultdict(int)
for i, ele in enumerate(labels):
    c=clustering_labels[i]
    if ele ==-1:
        abnormal[c]+=1
prob = defaultdict(float)
for n in range(n_cls):
    if total[n]==0:
        prob[n]=0
    else:
        prob[n] = round(abnormal[n]/total[n], 3)
pickle.dump(prob, open('risk_model/prob.pkl', 'wb'))